## Aula 05 - Validação de modelo


In [ ]:
import numpy as np
import pandas as pd

# mais informações desse dataset: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
chuvas = pd.read_csv('weatherAUS.csv')
chuvas = chuvas.sample(5000)

print(chuvas.shape)

In [ ]:
chuvas.describe()

In [ ]:
data = chuvas[['MinTemp','MaxTemp','Rainfall']]
data = data.fillna({'MinTemp':data.MinTemp.mean(),'MaxTemp':data.MaxTemp.mean(),'Rainfall':data.Rainfall.mean()})

target = chuvas[['RainTomorrow']].values.ravel()

### Hold-out

In [ ]:
# separando os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [ ]:
# treinando o modelo 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# avaliando o modelo
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

### Grid Search

In [ ]:
from sklearn.model_selection import cross_val_score
knn = KNeighborsClassifier(n_neighbors = 1)
scores = cross_val_score(knn, data, target, cv=5) # 5 execuções diferentes com 20% dos dados para teste

print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

In [ ]:
# utilizando validação cruzada com KFold
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data, target): # precisa passar as classes agora para que a divisão aconteça
    
    # etapa de treinamento
    knn = KNeighborsClassifier(n_neighbors = 1)
    knn.fit(data.values[train_index],target[train_index])
    
    # etapa de teste
    y_pred = knn.predict(data.values[test_index])
    acc.append(accuracy_score(y_pred,target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

#### Manipulando as etapas de treino e teste individualmente

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

kf = StratifiedKFold(n_splits = 5)

acc = []
for train_index, test_index in kf.split(data, target): # precisa passar as classes agora para que a divisão aconteça
    knn = KNeighborsClassifier(n_neighbors = 1)
    
    scaler = StandardScaler()
    train = scaler.fit_transform(data.values[train_index]) # somente dados de treino no fit
    test = scaler.transform(data.values[test_index]) # aplica-se transform no teste apenas
    
    knn.fit(train,target[train_index])
    y_pred = knn.predict(test)
    acc.append(accuracy_score(y_pred, target[test_index]))

acc = np.asarray(acc) # converte pra numpy pra ficar mais simples de usar média e desvio padrão
print('Acurácia - %.2f +- %.2f' % (acc.mean() * 100, acc.std() * 100))

#### Utilizando Pipeline do Scikit-Learn

In [ ]:
# utilizando validação cruzada com cross_val_score
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier(n_neighbors = 1))])
scores = cross_val_score(pipeline, data, target, cv=5) # 5 execuções diferentes com 20% dos dados para teste

print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

### Procurando por hiperparâmetros

In [ ]:
# separa-se uma parcela para encontrar os melhores parâmetros (5% do original)
data_gs, data_cv, target_gs, target_cv = train_test_split(data, target, test_size=0.95, random_state=42, stratify=target)

# uma forma automática de StandardScaler + CLF
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])

# utiliza-se GridSearchCV para achar os melhores parâmetros
from sklearn.model_selection import GridSearchCV
parameters = {'clf__n_neighbors': [1,2,3,4,5], 'clf__weights' : ['uniform','distance']} # quais parâmetros e quais valores serão testados
clf = GridSearchCV(pipeline, parameters, cv=3) # clf vai armazenar qual foi a melhor configuração
clf.fit(data_gs, target_gs)

print(clf.best_params_)

# utilizando validação cruzada para avaliar o modelo
scores = cross_val_score(clf.best_estimator_, data_cv, target_cv, cv=5)
print('Resultados:', scores)
print('Acurácia - %.2f +- %.2f' % (scores.mean() * 100, scores.std() * 100))

# Exercício

Construir um modelo avaliando os aspectos estudados até agora:

- Fazer análise das colunas (considerando todas);
- Escolher as colunas a serem utilizadas (exceção de RainTomorrow);
- Decidir qual pré-processamento será feito;
- Aplicar a busca por parâmetros;
- Validar o modelo usando validação cruzada;
- Apresentar os resultados.